In [84]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import pandas as pd
from pandas import DataFrame
from pandas import Series
from netCDF4 import Dataset
from netCDF4 import num2date
import datetime
import os
import sys
import os.path
import glob

In [85]:
# Reading in model output files, from three different applications, fabm0d, gotm1d and gotmlake
output_dir= os.path.normpath(os.getcwd() + os.sep + os.pardir+"/output")
# fabm0d files
fabm0d=[]
for f in glob.glob("*fabm0d*.nc"):
    fabm0d.append(f)
# gotm1d files
gotm1d=[]
for f in glob.glob("*gotm1d*.nc"):
    gotm1d.append(f)
# gotm-lake files
gotmlake=[]
for f in glob.glob("*gotmlake*.nc"):
    gotmlake.append(f)

In [86]:
# Read in gotm1d data, get the vertical avarage value, and store in nc Dataset form
# creat dict for store output of wanted fabm0d data
df_fabm0d={}
for f in fabm0d:
    path=os.path.join(output_dir, f)
    fabm0d_nc=Dataset(path, mode='r')
    time        = fabm0d_nc.variables['time']
    units       = fabm0d_nc.variables['time'].units
    valid_times = num2date(time[:], units=units).tolist()
# define the start of the time and the end of the time for plot range
    start=valid_times.index(datetime.datetime(2015, 1, 1, 0, 0))
    stop=valid_times.index(datetime.datetime(2016, 1, 1, 0, 0))
    time=valid_times[start:stop]
    # create empty lists for storging extracted and treated variables
    temperature=[]
    PAR=[]
    Oxygen=[]
    TP=[]
    TN=[]
    Phyto=[]
    Zoo=[]
    Fish=[] 
    Veg=[]
    Ben=[]
# loop over time and put the variable in lists
    i=0
    for t in time:
        temperature.append(fabm0d_nc.variables['temp'][i,0,0])
        Oxygen.append(fabm0d_nc.variables['abiotic_water_sO2W'][i,0,0])
        TP.append(fabm0d_nc.variables['pclake_totP_calculator_result'][i,0,0])
        TN.append(fabm0d_nc.variables['pclake_totN_calculator_result'][i,0,0])
        Phyto.append(fabm0d_nc.variables['phytoplankton_water_aDPhytW'][i,0,0])
        Zoo.append(fabm0d_nc.variables['foodweb_water_sDZoo'][i,0,0])
        Fish.append(fabm0d_nc.variables['foodweb_water_sDFiAd'][i,0,0]+
                    fabm0d_nc.variables['foodweb_water_sDFiJv'][i,0,0])
        PAR.append(fabm0d_nc.variables['phytoplankton_water_partop'][i,0,0])
        Veg.append(fabm0d_nc.variables['macrophytes_sDVeg'][i,0,0])
        Ben.append(fabm0d_nc.variables['foodweb_sediment_sDBent'][i,0,0])
        i=i+1
# put list into pandas dataframe format
    temp=DataFrame(temperature,index=time,columns=['temp'],)
    PAR=DataFrame(PAR,index=time,columns=['PAR'])
    O2=DataFrame(Oxygen,index=time,columns=['O2'])
    totP=DataFrame(TP,index=time,columns=['totP'])
    totN=DataFrame(TN,index=time,columns=['totN'])
    aDPhytW=DataFrame(Phyto,index=time,columns=['aDPhytW'])
    sDZoo=DataFrame(Zoo,index=time,columns=['sDZoo'])
    aDFish=DataFrame(Fish,index=time,columns=['aDFish'])
    sDVeg=DataFrame(Veg,index=time,columns=['sDVeg'])
    sDBent=DataFrame(Ben,index=time,columns=['sDBent'])   
# put data into dict for future usage
    df_fabm0d[f]=pd.concat([temp,PAR,O2,totP,totN,aDPhytW,sDZoo,aDFish,sDVeg,sDBent],axis=1)
    

In [87]:
# Read in gotm1d data, get the vertical avarage value, and store in nc Dataset form
# creat dict for store output of wanted GOTM1D data
df_gotm1d={}
# Loop over gotm1d files and get the netcdf dataa
for f in gotm1d:
    path=os.path.join(output_dir, f)
    gotm1d_nc=Dataset(path, mode='r')
    time        = gotm1d_nc.variables['time']
    units       = gotm1d_nc.variables['time'].units
    valid_times = num2date(time[:], units=units).tolist()
    time=valid_times[start:stop]
# create empty lists for storging extracted and treated variables
    temperature=[]
    PAR=[]
    Oxygen=[]
    TP=[]
    TN=[]
    Phyto=[]
    Zoo=[]
    Fish=[] 
    Veg=[]
    Ben=[]
# loop over time and put the variable in lists
    i=0
    for t in time:
        temperature.append(np.mean(gotm1d_nc.variables['temp'][i,:,0,0]))
        Oxygen.append(np.mean(gotm1d_nc.variables['abiotic_water_sO2W'][i,:,0,0]))
        TP.append(np.mean(gotm1d_nc.variables['pclake_totP_calculator_result'][i,:,0,0]))
        TN.append(np.mean(gotm1d_nc.variables['pclake_totN_calculator_result'][i,:,0,0]))
        Phyto.append(np.mean(gotm1d_nc.variables['phytoplankton_water_aDPhytW'][i,:,0,0]))
        Zoo.append(np.mean(gotm1d_nc.variables['foodweb_water_sDZoo'][i,:,0,0]))
        Fish.append(np.mean(gotm1d_nc.variables['foodweb_water_sDFiAd'][i,:,0,0])+
                    np.mean(gotm1d_nc.variables['foodweb_water_sDFiJv'][i,:,0,0]))
        PAR.append(gotm1d_nc.variables['phytoplankton_water_partop'][i,-1,0,0])
        Veg.append(gotm1d_nc.variables['macrophytes_sDVeg'][i,0,0])
        Ben.append(gotm1d_nc.variables['foodweb_sediment_sDBent'][i,0,0])
        i=i+1
# put list into pandas dataframe format
    temp=DataFrame(temperature,index=time,columns=['temp'],)
    PAR=DataFrame(PAR,index=time,columns=['PAR'])
    O2=DataFrame(Oxygen,index=time,columns=['O2'])
    totP=DataFrame(TP,index=time,columns=['totP'])
    totN=DataFrame(TN,index=time,columns=['totN'])
    aDPhytW=DataFrame(Phyto,index=time,columns=['aDPhytW'])
    sDZoo=DataFrame(Zoo,index=time,columns=['sDZoo'])
    aDFish=DataFrame(Fish,index=time,columns=['aDFish'])
    sDVeg=DataFrame(Veg,index=time,columns=['sDVeg'])
    sDBent=DataFrame(Ben,index=time,columns=['sDBent'])
# put data into dict for future usage
    df_gotm1d[f]=pd.concat([temp,PAR,O2,totP,totN,aDPhytW,sDZoo,aDFish,sDVeg,sDBent],axis=1)


In [88]:
# Read in gotm1d data, store in nc Dataset form
# Read in gotm1d_2m
# Create empty dict for storing output of wanted gotmlake data
df_gotmlake={}
# Get the volumn fraction for each depth in gotmlake
# calculated according to hypsograph data in excel.
f_Vn_2m=[0.04,0.12,0.20,0.28,0.36]
f_Vn_5m=[0.01,0.02,0.03,0.04,0.05,0.07,0.08,0.09,0.10,0.11,0.12,0.13,0.15]
f_Vn_10m=[0.0025,0.0056,0.0087,0.0119,0.0150,0.0181,0.0212,0.0244,0.0275,
          0.0306,0.0337,0.0369,0.0400,0.0431,0.0463,0.0494,0.0525,0.0556,
          0.0588,0.0619,0.0650,0.0681,0.0713,0.0744,0.0775]
f_Vn_20m=[0.000854443,0.001635895,0.002417346,0.003198797,0.003980249,0.0047617,
          0.005543151,0.006324602,0.007106054,0.007887505,0.008668956,0.009450408,
          0.010231859,0.01101331,0.011794761,0.012576213,0.013357664,0.014139115,
          0.014920567,0.015702018,0.016483469,0.01726492,0.018046372,0.018827823,
          0.019609274,0.020390726,0.021172177,0.021953628,0.02273508,0.023516531,
          0.024297982,0.025079433,0.025860885,0.026642336,0.027423787,0.028205239,
          0.02898669,0.029768141,0.030549592,0.031331044,0.032112495,0.032893946,
          0.033675398,0.034456849,0.0352383,0.036019751,0.036801203,0.037582654,
          0.038364105,0.039145557]
# Loop over gotmlake files and get the netcdf data
for f in gotmlake:
    path=os.path.join(output_dir, f)
    gotmlake_nc=Dataset(path, mode='r')
    time        = gotmlake_nc.variables['time']
    units       = gotmlake_nc.variables['time'].units
    valid_times = num2date(time[:], units=units).tolist()
    time=valid_times[start:stop]
# Get the f_lvl for different depth
    lvl=len(gotmlake_nc.variables['temp'][0])
    if lvl==5:
        f_lvl=f_Vn_2m
    elif lvl== 13:
        f_lvl=f_Vn_5m
    elif lvl==25:
        f_lvl=f_Vn_10m
    elif lvl==50:
        f_lvl=f_Vn_20m
    else:
        print "I can't find f_lvl"      
# create empty lists for storging extracted and treated variables
    temperature=[]
    PAR=[]
    Oxygen=[]
    TP=[]
    TN=[]
    Phyto=[]
    Zoo=[]
    Fish=[] 
    Veg=[]
    Ben=[]
# Loop over time, treat the data and put variables in lists
    i=0
    for t in time:      
# get the volume average data from gotmlake
        temperature.append(np.sum(gotmlake_nc.variables['temp'][i,:,0,0]*f_lvl[:]))
        Oxygen.append(np.sum(gotmlake_nc.variables['abiotic_water_sO2W'][i,:,0,0]*f_lvl[:]))
        TP.append(np.sum(gotmlake_nc.variables['pclake_totP_calculator_result'][i,:,0,0]*f_lvl[:]))
        TN.append(np.sum(gotmlake_nc.variables['pclake_totN_calculator_result'][i,:,0,0]*f_lvl[:]))
        Phyto.append(np.sum(gotmlake_nc.variables['phytoplankton_water_aDPhytW'][i,:,0,0]*f_lvl[:]))
        Zoo.append(np.sum(gotmlake_nc.variables['foodweb_water_sDZoo'][i,:,0,0]*f_lvl[:]))
        Fish.append(np.sum(gotmlake_nc.variables['foodweb_water_sDFiAd'][i,:,0,0]*f_lvl[:])+
                           np.sum(gotmlake_nc.variables['foodweb_water_sDFiJv'][i,:,0,0]*f_lvl[:]))
# PAR is stil the top layer data
        PAR.append(gotmlake_nc.variables['phytoplankton_water_partop'][i,-1,0,0])
# Get gotmlake sediment data, now it's 1D for state variables
        Veg.append(np.mean(gotmlake_nc.variables['macrophytes_sDVeg'][i,:,0,0]))
        Ben.append(np.mean(gotmlake_nc.variables['foodweb_sediment_sDBent'][i,:,0,0]))
        i=i+1
# Convert data into pandas dataframework
    temp=DataFrame(temperature,index=time,columns=['temp'],)
    PAR=DataFrame(PAR,index=time,columns=['PAR'])
    O2=DataFrame(Oxygen,index=time,columns=['O2'])
    totP=DataFrame(TP,index=time,columns=['totP'])
    totN=DataFrame(TN,index=time,columns=['totN'])
    aDPhytW=DataFrame(Phyto,index=time,columns=['aDPhytW'])
    sDZoo=DataFrame(Zoo,index=time,columns=['sDZoo'])
    aDFish=DataFrame(Fish,index=time,columns=['aDFish'])
    sDVeg=DataFrame(Veg,index=time,columns=['sDVeg'])
    sDBent=DataFrame(Ben,index=time,columns=['sDBent'])
# Store gotmlake data into dict for future usage
    df_gotmlake[f]=pd.concat([temp,PAR,O2,totP,totN,aDPhytW,sDZoo,aDFish,sDVeg,sDBent],axis=1)

In [89]:
# join dataframe according to depths
df_2m= pd.concat([df_fabm0d['pclake-fabm0d-2m.nc'],df_gotm1d['pclake-gotm1d-2m.nc'],
                  df_gotmlake['pclake-gotmlake-2m.nc']],axis=1,keys=['fabm0d', 'gotm1d', 'gotmlake'])
df_5m= pd.concat([df_fabm0d['pclake-fabm0d-5m.nc'],df_gotm1d['pclake-gotm1d-5m.nc'],
                  df_gotmlake['pclake-gotmlake-5m.nc']],axis=1,keys=['fabm0d', 'gotm1d', 'gotmlake'])
df_10m= pd.concat([df_fabm0d['pclake-fabm0d-10m.nc'],df_gotm1d['pclake-gotm1d-10m.nc'],
                   df_gotmlake['pclake-gotmlake-10m.nc']],axis=1,keys=['fabm0d', 'gotm1d', 'gotmlake'])
df_20m= pd.concat([df_fabm0d['pclake-fabm0d-20m.nc'],df_gotm1d['pclake-gotm1d-20m.nc'],
                   df_gotmlake['pclake-gotmlake-20m.nc']],axis=1,keys=['fabm0d', 'gotm1d', 'gotmlake'])
# Joined all data together for plot
results= pd.concat([df_2m,df_5m,df_10m,df_20m],axis=1,keys=['2m','5m','10m','20m'])

In [223]:
# Library for legends(only used here)
# draw lengend according to the upper right corner
# Variables for ploting legend
fontP = FontProperties()
fontP.set_size('xx-small')
# Plot line plots with three different model output
# Give the lables of variables for plotting
models=['fabm0d','gotm1d','gotmlake']
variables_group1 = ['temp', 'totN', 'totP',  'O2']
variables_group2= ['PAR','aDPhytW','sDZoo','aDFish']
depths=['2m','5m','10m','20m']
xticklabels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct',',Nov','Dec']
colors=['b','g','r']
lines=['--','-','-.']

# Formating ticks
#majorLocator = plt.MultipleLocator()
#majorFormatter = plt.FormatStrFormatter('%d')

# Plot group one variables
fig1 = plt.figure(figsize=(8.27, 11.69), dpi=1200)
fig1, axs = plt.subplots(4,4,sharex=True,squeeze=True, ) 
# adjust the space between subplots
fig1.subplots_adjust(hspace = 0.07)
#plt.subplots_adjust(hspace = .1)
j=0
for depth in depths:
    i=0
    for var in variables_group1:
        k=0
        for model in models:
            axs[j,i].plot(time,results[depth][model][var], color=colors[k],linestyle=lines[k],label=model)
            k=k+1
# Set x-axis and y-axis ticks
        axs[j,i].set_xticklabels(xticklabels,rotation='vertical',fontsize =5)
        for ytick in axs[j,i].yaxis.get_major_ticks():
            ytick.label.set_fontsize(5)
        i=i+1
    j=j+1
# loc[x,y], y is vertial positon, y, 1 is top
# x is horizontal positon, 1 is the right
# The following is setting legends and labels for the picture
# i.e making the picture look beautiful
# Make legend, accoring to subplot [2,3], place it on the right side of the figure
legend = axs[2,3].legend(loc=1, ncol=1, bbox_to_anchor=(0, 0, 1.65,1.4),
                         prop = fontP,fancybox=True,shadow=False)
legend.draw_frame(False)
plt.setp(legend.get_title(),fontsize='xx-small')
# Add text for variables
fig1.text(0.15,0.92,'Temperature',**{'fontsize':10})
fig1.text(0.38,0.92,'Oxygen',**{'fontsize':10})
fig1.text(0.55,0.92,'Total Nitrogen',**{'fontsize':10})
fig1.text(0.75,0.92,'Total Phosphorus',**{'fontsize':10})
# Add text for depth
fig1.text(0.05,0.8,'2m',**{'fontsize':10})
fig1.text(0.05,0.6,'5m',**{'fontsize':10})
fig1.text(0.05,0.4,'10m',**{'fontsize':10})
fig1.text(0.05,0.2,'20m',**{'fontsize':10})
# Save figure, and reduce the margins
fig1.savefig('fig1.png',bbox_inches='tight')
# plot group two variables
fig2 = plt.figure(figsize=(8.27, 11.69), dpi=1200)
fig2, axs = plt.subplots(4,4,sharex=True) 
j=0
for depth in depths:
    i=0
    for var in variables_group2:
        k=0
        for model in models:
            axs[j,i].plot(time,results[depth][model][var], color=colors[k],linestyle=lines[k],label=model)
            k=k+1
# Set x-axis and y-axis ticks
        axs[j,i].set_xticklabels(xticklabels,rotation='vertical',fontsize =5)
        for ytick in axs[j,i].yaxis.get_major_ticks():
            ytick.label.set_fontsize(5)
        axs[j,i].set_xticklabels(xticklabels,rotation='vertical',fontsize =5)
        for ytick in axs[j,i].yaxis.get_major_ticks():
            ytick.label.set_fontsize(5)
        i=i+1
    j=j+1
# The following is setting legends and labels for the picture
# i.e making the picture look beautiful
# Make legend, accoring to subplot [2,3], place it on the right side of the figure
legend = axs[2,3].legend(loc=1, ncol=1, bbox_to_anchor=(0, 0, 1.65,1.4),
                         prop = fontP,fancybox=True,shadow=False)
legend.draw_frame(False)
plt.setp(legend.get_title(),fontsize='xx-small')
# Add text for variables
fig2.text(0.19,0.92,'PAR',**{'fontsize':10})
fig2.text(0.35,0.92,'Phytoplankton',**{'fontsize':10})
fig2.text(0.56,0.92,'Zooplankton',**{'fontsize':10})
fig2.text(0.79,0.92,'Fish',**{'fontsize':10})
# Add text for depth
fig2.text(0.05,0.8,'2m',**{'fontsize':10})
fig2.text(0.05,0.6,'5m',**{'fontsize':10})
fig2.text(0.05,0.4,'10m',**{'fontsize':10})
fig2.text(0.05,0.2,'20m',**{'fontsize':10})
# Save figure, and reduce the margins
fig2.savefig('fig2.png',bbox_inches='tight')    

#possible code for surface output
if f[-6:-5]=='2m':
lvl=4
print lvl
PAR.append(gotm1d_nc.variables['rad'][i,lvl,0,0])
elif f[-6:-5]=='5m':
lvl=12
print lvl
PAR.append(gotm1d_nc.variables['rad'][i,lvl,0,0])
elif f[-6:-5]=='10m':
lvl=24
print lvl
PAR.append(gotm1d_nc.variables['rad'][i,lvl,0,0])
elif f[-6:-5]=='20m':
lvl=49
print lvl
PAR.append(gotm1d_nc.variables['rad'][i,lvl,0,0])